In [103]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.linear_model import Ridge, Lasso, ElasticNet
#import statsmodels.api as sm
import matplotlib.pyplot as plt
import itertools
import time


In [69]:
# 0/1 cut-off(임계값) 함수

def cut_off(y, threshold):
    Y = y.copy()
    Y[Y>threshold] = 2
    Y[Y<=threshold] = 1
    return (Y.astype(int))
    
# 정확도 acc 함수 

def acc(cfmat):
    acc = (cfmat[0,0] + cfmat[1,1]) / np.sum(cfmat)
    return acc

In [35]:
df = pd.read_csv("ckd.analysis0706final.csv")
df=df.dropna(subset = ["new.cardiovascular"])
exclude=['Unnamed: 0','new.cardiovascular','rand_num','Phase','p8_p6','p8_p6.PerYear','p8_p6.pctChangesPerYear']
feature_columns = list(df.columns.difference(exclude))
x = df[feature_columns]
y = df['new.cardiovascular']
trainx, testx, trainy, testy = train_test_split(x,y,stratify = y, train_size =0.7, test_size =0.3, random_state = 42)
print(trainx.shape, testx.shape, trainy.shape, testy.shape)

# Lasso

In [70]:
ll = Lasso(alpha = 0.01 ) # alpha = Lambda
ll.fit(trainx,trainy)

Lasso(alpha=0.01)

In [80]:
## 예측, confusion matrix, acc 계산

predyLasso = ll.predict(testx)

In [86]:
from scipy import stats
print(stats.describe(predyLasso)) ## probability가 출력되는게 아닌가? max가 1이상임..



DescribeResult(nobs=566, minmax=(0.8630117903528005, 1.5281258731069016), mean=1.0365830639083988, variance=0.0050095272934338264, skewness=1.062129764443186, kurtosis=4.884721589598355)


In [76]:

predyLasso = cut_off(predyLasso, 0.5)
cfmat = confusion_matrix(testy,predyLasso)
print(acc(cfmat))


0.037102473498233215


In [68]:
testy=testy.astype('int64', copy=False)
fpr,tpr, thresholds = metrics.roc_curve(testy,predyLasso)

# print ROC curve
plt.plot(fpr,tpr)

# print AUC 
auc = np.trapz(tpr, fpr)
print("AUC :",auc)

ValueError: y_true takes value in {1, 2} and pos_label is not specified: either make y_true take value in {0, 1} or {-1, 1} or pass pos_label explicitly.

# sklearn.linear_model.LogisticRegression

In [87]:
from sklearn.linear_model import LogisticRegression

In [102]:
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)  
#n_split: Number of folds. Must be at least 2;n_repeates: Number of times cross-validator needs to be repeated.

# define search
search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1) #scoring:accuracy, precision..

# perform the search
results = search.fit(X, y)
# summarize
print('Accuracy: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

NameError: name 'RepeatedKFold' is not defined

In [ ]:
grid_search.score(X_test, y_test)

In [127]:
#SAGA는 SAGStochastic Average Gradient 알고리즘의 변종, 혹은 개선 버전입니다
#(SAGA의 A가 특별한 의미가 있는 것은 아닙니다). 
#SAG 알고리즘은 이전 타임스텝에서 계산된 그래디언트를 모두 평균내어 적용하는 알고리즘입니다. 
#기본 공식은 일반적인 SGDStochastic Gradient Descent과 비슷합니다. 
#다만 이전에 계산했던 그래디언트를 저장하고 다시 활용한다는 측면에서 
#SGD와 배치 그래디언트 디센트의 장점을 취하고 있습니다.
#https://tensorflow.blog/2017/12/15/new-saga-solver/comment-page-1/
#SAGA 알고리즘은 Ridge, RidgeClassifier, LogisticRegression 등에 solver 매개변수를 ‘saga’로 
#설정하여 사용할 수 있습니다. 
#이 모델들은 대량의 데이터셋에서 SAG 알고리즘을 사용할 수 있었는데 
#SAGA가 SAG 보다 성능이 좋으므로 
#데이터셋이 클 경우 SAGA를 항상 사용하는 것이 좋을 것 같습니다.



# 파라메터 후보
param_grid = {'C': [0.001,0.01, 0.1, 1, 10, 100], #0.001,
              #'alpha':[np.arange(0,1,0.01)],
              'penalty': ['l1', 'l2'],
              'solver':['saga'],
              'verbose':[1]
             }
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)  
#n_split: Number of folds. Must be at least 2;n_repeates: Number of times cross-validator needs to be repeated.

# 그리드 서치 진행
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=cv)        


In [128]:
#LogisticRegression().get_params().keys()
results = grid_search.fit(trainx, trainy)
print('Accuracy: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 0 seconds


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 1 seconds


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 1 seconds


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 0 seconds
Accuracy: 0.964
Config: {'C': 0.001, 'penalty': 'l1', 'solver': 'saga', 'verbose': 1}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished


In [131]:
res=pd.DataFrame(grid_search.cv_results_)
res[['split0_test_score', 'split1_test_score', 'split2_test_score','split3_test_score', 'split4_test_score', 'mean_test_score']]
print(res[['params', 'mean_test_score']])  

                                               params  mean_test_score
0   {'C': 0.001, 'penalty': 'l1', 'solver': 'saga'...         0.963609
1   {'C': 0.001, 'penalty': 'l2', 'solver': 'saga'...         0.963609
2   {'C': 0.01, 'penalty': 'l1', 'solver': 'saga',...         0.963609
3   {'C': 0.01, 'penalty': 'l2', 'solver': 'saga',...         0.963609
4   {'C': 0.1, 'penalty': 'l1', 'solver': 'saga', ...         0.963609
5   {'C': 0.1, 'penalty': 'l2', 'solver': 'saga', ...         0.963609
6   {'C': 1, 'penalty': 'l1', 'solver': 'saga', 'v...         0.963609
7   {'C': 1, 'penalty': 'l2', 'solver': 'saga', 'v...         0.963609
8   {'C': 10, 'penalty': 'l1', 'solver': 'saga', '...         0.963609
9   {'C': 10, 'penalty': 'l2', 'solver': 'saga', '...         0.963609
10  {'C': 100, 'penalty': 'l1', 'solver': 'saga', ...         0.963609
11  {'C': 100, 'penalty': 'l2', 'solver': 'saga', ...         0.963609


In [132]:
best_model = grid_search.best_estimator_
y_predict = best_model.predict(testx)


NameError: name 'labels' is not defined

In [201]:
best_model

LogisticRegression(C=0.001, penalty='l1', solver='saga', verbose=1)

In [166]:
temp=[]
temp.extend('a')

['a']

In [168]:
y_predict = best_model.predict_proba(testx) #predict_proba의 출력은 항상 0과 1 사이의 값이며 두 클래스에 대한 확률의 합은 항상 1
print(stats.describe(y_predict)) ##
y_trueProb=[]
for prob in y_predict:
    y_trueProb.append(prob[1])

In [190]:
 # keep probabilities for the positive outcome only
lr_probs = y_predict[:, 1]
lr_probs

array([0.01095672, 0.01428995, 0.02151265, 0.02512928, 0.00827732,
       0.06030168, 0.03992259, 0.02044514, 0.01517734, 0.04868896,
       0.01601392, 0.13425389, 0.01535686, 0.01685279, 0.02785394,
       0.02418989, 0.03199177, 0.02806529, 0.07823896, 0.02855125,
       0.02037049, 0.08708712, 0.03025269, 0.01771639, 0.02042121,
       0.0112481 , 0.01391065, 0.09748112, 0.02574785, 0.03199177,
       0.04436344, 0.03184678, 0.0509155 , 0.02175933, 0.01478154,
       0.03972815, 0.02030091, 0.0210675 , 0.03302216, 0.07789517,
       0.05912031, 0.03492844, 0.00738969, 0.05605948, 0.07572579,
       0.07190031, 0.02035094, 0.01397521, 0.01511186, 0.02151439,
       0.01491718, 0.02091061, 0.09085661, 0.00823216, 0.01649917,
       0.04612129, 0.02903642, 0.06030168, 0.09392379, 0.00418861,
       0.02106106, 0.00942732, 0.07928902, 0.03211795, 0.01470101,
       0.04598496, 0.04547415, 0.0418205 , 0.01379949, 0.03929775,
       0.10899338, 0.02010085, 0.01297054, 0.07349453, 0.03128

In [192]:
y_predictTF = cut_off(pd.DataFrame(y_trueProb), 0.5)
y_predictTF

,0
0,1
1,1
2,1
3,1
4,1
...,...
561,1
562,1
563,1
564,1


In [191]:
lr_probsTF=cut_off(pd.DataFrame(y_trueProb), 0.5)
lr_probsTF

,0
0,1
1,1
2,1
3,1
4,1
...,...
561,1
562,1
563,1
564,1


In [186]:
## 예측, confusion matrix, acc 계산
confusionMatrix = confusion_matrix(testy,y_predictTF)
print(acc(cfmat))
print(confusionMatrix)

0.037102473498233215
[[545   0]
 [ 21   0]]


In [200]:
fpr,tpr,threshold = metrics.roc_curve(testy,y_predictTF,pos_label=2)

lr_auc = roc_auc_score(testy, lr_probs)

# print ROC curve
#plt.plot(fpr,tpr)

# print AUC 
#auc = np.trapz(tpr, fpr)
print("AUC :",lr_auc)

AUC : 0.6283966797728265
